In [18]:
import pandas as pd
from pandas.io.common import EmptyDataError
from tqdm.autonotebook import trange, tqdm

import chart_studio.plotly as py
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from datetime import datetime, timedelta, timezone
from pytz import timezone
import humanize

import os
import shutil
import re

import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import time
import numpy as np
import math
import seaborn as sns

import requests
import io
import warnings

from collections import OrderedDict
from PyPDF2 import PdfFileMerger, PdfFileReader, PdfFileWriter

In [19]:
def import_df(filename, index = False):
    try:
        if index == False:
            df = pd.read_csv(filename)
        else:
            df = pd.read_csv(filename, index_col=0)
    except EmptyDataError:
        df = pd.DataFrame()
    return df

def set_date(date_str, old_tz, dt_format = "%d/%m/%Y %H:%M:%S"):
    if date_str == 'NaT':
        return pd.NaT
    else:
        datetime_set_naive = datetime.strptime(date_str, dt_format)
        datetime_set_old = timezone(old_tz).localize(datetime_set_naive)
        datetime_set_utc = datetime_set_old.astimezone(timezone('UTC'))
        return datetime_set_utc

In [20]:
info = OrderedDict()
info['setup'] = import_df("Temp/Data/info/info_setup.csv", index = True)
date_end = set_date(info['setup'].loc['date_end_utc','value'].replace('+00:00',''), "UTC", "%Y-%m-%d %H:%M:%S.%f")
diff = datetime.now(timezone('UTC')) - date_end
last_date = humanize.naturaldelta(diff)

In [21]:
if diff > timedelta(hours = 4):
    from Scripts import GetLatestData

0%|███       | 102/340 [00:06<00:14, 16.03it/s]



Open files to import:  31%|███       | 104/340 [00:06<00:14, 16.23it/s]



Open files to import:  31%|███       | 106/340 [00:06<00:14, 16.24it/s]



Open files to import:  32%|███▏      | 108/340 [00:06<00:14, 16.33it/s]



Open files to import:  32%|███▏      | 110/340 [00:06<00:14, 16.32it/s]



Open files to import:  33%|███▎      | 112/340 [00:07<00:13, 16.62it/s]



Open files to import:  34%|███▎      | 114/340 [00:07<00:13, 16.87it/s]



Open files to import:  34%|███▍      | 116/340 [00:07<00:13, 16.87it/s]



Open files to import:  35%|███▍      | 118/340 [00:07<00:13, 16.69it/s]



Open files to import:  35%|███▌      | 120/340 [00:07<00:13, 16.77it/s]



Open files to import:  36%|███▌      | 122/340 [00:07<00:13, 16.71it/s]



Open files to import:  36%|███▋      | 124/340 [00:07<00:12, 17.11it/s]



Open files to import:  37%|███▋      | 126/340 [00:07<00:12, 16.96it/s]



Open files to import:  38%|███▊      | 128/340 [0

In [22]:
#Import data from temp_files
info = OrderedDict()
chart_dfs_mlt = []
plot_pars = []

data_folder = "Temp/Data/"

for folder in tqdm(os.listdir(data_folder), desc = "Open files to import"):
    if folder.startswith('info'):
        for filename in os.listdir(data_folder + "info"):
            file_id = filename.replace('info_', '')
            file_id = file_id.replace('.csv', '')
            info[file_id] = import_df(data_folder + "info/" + filename, index = True)
    if folder.startswith('chart_dfs_mlt'):
        for filename in os.listdir(data_folder + "chart_dfs_mlt"):
            chart_dfs_mlt.append(import_df(data_folder + "chart_dfs_mlt/" + filename))
    if folder.startswith('plot_pars'):
        for filename in os.listdir(data_folder + "plot_pars"):
            plot_pars.append(import_df(data_folder + "plot_pars/" + filename, index = True))

charts = info['charts'].index.to_list()
date_end = set_date(info['setup'].loc['date_end_utc','value'].replace('+00:00',''), "UTC", "%Y-%m-%d %H:%M:%S.%f")




Open files to import:   0%|          | 0/3 [00:00<?, ?it/s]


Open files to import: 100%|██████████| 3/3 [00:00<00:00,  3.49it/s]


In [23]:
chart_figs = []
for chart in charts:
    if info['charts'].loc[chart, 'chart_status'] == 'ON':
        chart_figs.append({})
        selected_plot_info = info['plots'].loc[info['plots'].index == chart][info['plots'].loc[info['plots'].index == chart]['plot'].isin(chart_dfs_mlt[chart].Plot.unique().tolist())]
        # For each plot
        for plot in tqdm(selected_plot_info['plot'].to_list(), desc = "Creating plots for chart " + str(chart)):
            
            fig_par = info['plots'].loc[info['plots'].index == chart].query('plot == "' + plot + '"')
            fig = go.Figure()
            
            #Add traces
            for i in range(0, len(chart_dfs_mlt[chart].query('Plot == "' + plot + '"').Parameter.unique())):
                par = chart_dfs_mlt[chart].query('Plot == "' + plot + '"').Parameter.unique()[i]
                plot_par = plot_pars[chart].query('parameter == "' + par + '"')
                if len(plot_par) != 0:
                    x_data = chart_dfs_mlt[chart][chart_dfs_mlt[chart].Parameter == par].DateTime
                    y_data = chart_dfs_mlt[chart][chart_dfs_mlt[chart].Parameter == par].Value
                    y_error = chart_dfs_mlt[chart][chart_dfs_mlt[chart].Parameter == par].Error

                    if plot_par['line'].values == True:
                        if plot_par['show_in_legend'].values == True:
                            if plot_par['point'].values == False and plot_par['bar'].values == False:
                                legend_show = True
                            else:
                                legend_show = False
                        else:
                            legend_show = False
                        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines',
                                                name=plot_par['parameter_lab'][0], 
                                                line=dict(color=plot_par['colour'][0], width=2,
                                                    dash=plot_par['dash'][0]),
                                                connectgaps=False,
                                                legendgroup=plot_par['parameter_lab'][0],
                                                showlegend=legend_show
                                                ))
                    if plot_par['point'].values == True:
                        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='markers',
                                                name=plot_par['parameter_lab'][0], 
                                                marker=dict(color=plot_par['fill'][0],
                                                symbol=plot_par['shape'][0],
                                                line=dict(color=plot_par['colour'][0],width=1)),
                                                connectgaps=True,
                                                legendgroup=plot_par['parameter_lab'][0],
                                                showlegend=bool(plot_par['show_in_legend'][0]),
                                                error_y=dict(
                                                    type='data',
                                                    array=y_error,
                                                    visible=True)
                                                ))
                    if plot_par['ribbon'].values == True:
                        fig.add_trace(go.Scatter(x=x_data, y=y_data + y_error, mode='lines',
                                                name=plot_par['parameter_lab'][0],
                                                line=dict(color=plot_par['colour'][0], width=0, dash = 'dot'),
                                                connectgaps=True,
                                                legendgroup=plot_par['parameter_lab'][0],
                                                showlegend=False,
                                                hoverinfo='skip'
                                                ))
                        fig.add_trace(go.Scatter(x=x_data, y=y_data - y_error, fill='tonexty', mode='none',
                                                fillcolor=plot_par['fill'][0],
                                                name=plot_par['parameter_lab'][0], 
                                                line=dict(color=plot_par['colour'][0], width=0.5, dash = 'dot'),
                                                connectgaps=True,
                                                legendgroup=plot_par['parameter_lab'][0],
                                                showlegend=False,
                                                hoverinfo='skip'
                                                )) # fill to trace0 y

            #Modify plot
            fig.update_layout(
                margin=dict(l=100, r=220, b=15, t=15, pad=10),
                template="simple_white",
                paper_bgcolor='rgba(0,0,0,0)',
                font=dict(
                    family="Arial",
                    size=12,
                    color="black"
                ))
            fig.update_yaxes(title_text=fig_par['ylab'][chart], mirror=True)
            fig.update_xaxes(showgrid=True, showticklabels=False, ticks="",
                showline=True, mirror=True,
                range=[min(chart_dfs_mlt[chart].DateTime), max(chart_dfs_mlt[chart].DateTime)],
                fixedrange=True) #prevent x zoom
            
            #Special plot mods
            if fig_par['log'][chart] == True:
                fig.update_layout(yaxis_type="log")
                fig.update_yaxes(range=[math.log(fig_par['ymin'][chart], 10), math.log(fig_par['ymax'][chart], 10)])
            else:
                fig.update_yaxes(range=[fig_par['ymin'][chart], fig_par['ymax'][chart]])

            if plot == selected_plot_info['plot'].to_list()[len(selected_plot_info['plot'].to_list())-1]:
                fig.update_xaxes(showticklabels=True, ticks="outside")

            chart_figs[chart][plot] = fig
    else:
        chart_figs.append("")

#del chart, fig, i, plot, par, plot_par, x_data, y_data, y_error




Creating plots for chart 0:   0%|          | 0/10 [00:00<?, ?it/s]


Creating plots for chart 0:  10%|█         | 1/10 [00:00<00:05,  1.53it/s]


Creating plots for chart 0:  20%|██        | 2/10 [00:01<00:05,  1.48it/s]


Creating plots for chart 0:  30%|███       | 3/10 [00:02<00:05,  1.25it/s]


Creating plots for chart 0:  40%|████      | 4/10 [00:03<00:04,  1.24it/s]


Creating plots for chart 0:  50%|█████     | 5/10 [00:04<00:03,  1.25it/s]


Creating plots for chart 0:  60%|██████    | 6/10 [00:04<00:02,  1.47it/s]


Creating plots for chart 0:  70%|███████   | 7/10 [00:05<00:02,  1.48it/s]


Creating plots for chart 0:  80%|████████  | 8/10 [00:05<00:01,  1.76it/s]


Creating plots for chart 0:  90%|█████████ | 9/10 [00:05<00:00,  2.01it/s]


Creating plots for chart 0: 100%|██████████| 10/10 [00:07<00:00,  1.40it/s]



Creating plots for chart 1:   0%|          | 0/12 [00:00<?, ?it/s]


Creating plots for chart 1:   8%|▊         | 1/12 [00:00<00:06,  1.68it/s]


Creating p

In [24]:
dcc_chart_figs = []
for chart in charts:
    if info['charts'].loc[chart, 'chart_status'] == 'ON':
        dcc_chart_figs.append([])
        p = 0
        for plot in chart_figs[chart]:
            if p != len(chart_figs[chart])-1:
                dcc_chart_figs[chart].append(
                    dcc.Graph(id='graph' + str(p),
                        figure=chart_figs[chart][plot],
                        style={'width': '98vw', 'height': '20vh'})
                    )
            else:
                dcc_chart_figs[chart].append(
                    dcc.Graph(id='graph' + str(p),
                        figure=chart_figs[chart][plot],
                        style={'width': '98vw', 'height': '25vh'})
                    )
            p = p + 1
    else:
        dcc_chart_figs.append("")

#del chart, p, plot

In [25]:
def delete_folder_contents(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

def combine(pdf_dir, output_pdf):
    merger = PdfFileMerger(strict=False)

    for filename in os.listdir(pdf_dir):
        merger.append(PdfFileReader(pdf_dir + filename, strict=False))

    with open(output_pdf, 'wb') as fh:
        merger.write(fh)

In [26]:
if not os.path.exists("Temp"):
    os.mkdir("Temp")
if not os.path.exists("Temp/PDFs"):
    os.mkdir("Temp/PDFs")
else:
    delete_folder_contents("Temp/PDFs")

for chart in tqdm(charts, desc="Exporting PDFs"):
    if info['charts'].loc[chart, 'chart_status'] == 'ON':
        folder = "Temp/PDFs/" + str(chart) + "_" + info['charts'].loc[chart, 'chart'] + "/"
        if not os.path.exists(folder):
            os.mkdir(folder)
        else:
            delete_folder_contents(folder)
        p = 0
        #Export individual pdfs
        for plot in chart_figs[chart]:
            chart_figs[chart][plot].write_image(folder + str(p).zfill(2) + "_" + plot + ".pdf", width=210*6, height=29.7*6, scale=1)
            p = p + 1
        
        #Combine pdfs
        if __name__ == '__main__':
            combine(pdf_dir=folder, 
                    output_pdf=folder + "all_pages.pdf")
        
        #Create combined pdf on one page
        with open(folder + 'all_pages.pdf', 'rb') as input_file:
            # load input pdf
            input_pdf = PdfFileReader(input_file)
            num_pages = input_pdf.getNumPages()
            output_pdf = input_pdf.getPage(num_pages-1)

            for p in tqdm(reversed(range(0,num_pages-1)), desc="Combining plots into one pdf:"):
                second_pdf = input_pdf.getPage(p)
                # dimensions for offset from loaded page (adding it to the top)
                offset_x = 0 # use for x offset -> output_pdf.mediaBox[2]
                offset_y = output_pdf.mediaBox[3]
                #merge pdf pages
                output_pdf.mergeTranslatedPage(second_pdf, offset_x, offset_y, expand=True)

            # write finished pdf
            with open("Output/" + str(chart) + "_" + info['charts'].loc[chart, 'chart'] + ".pdf", 'wb') as out_file:
                    write_pdf = PdfFileWriter()
                    write_pdf.addPage(output_pdf)
                    write_pdf.write(out_file)
        
        #Delete temp pdfs
        try:
            shutil.rmtree(folder)
        except OSError as e:
            print ("Error: %s - %s." % (e.filename, e.strerror))




Exporting PDFs:   0%|          | 0/5 [00:00<?, ?it/s]



Combining plots into one pdf:: 0it [00:00, ?it/s]



Combining plots into one pdf:: 1it [00:01,  1.36s/it]



Combining plots into one pdf:: 2it [00:01,  1.04s/it]



Combining plots into one pdf:: 3it [00:02,  1.09s/it]



Combining plots into one pdf:: 4it [00:04,  1.13s/it]



Combining plots into one pdf:: 5it [00:05,  1.22s/it]



Combining plots into one pdf:: 6it [00:07,  1.31s/it]



Combining plots into one pdf:: 7it [00:08,  1.29s/it]



Combining plots into one pdf:: 8it [00:10,  1.43s/it]



Combining plots into one pdf:: 9it [00:11,  1.27s/it]



Exporting PDFs:  20%|██        | 1/5 [00:18<01:12, 18.24s/it]



Combining plots into one pdf:: 0it [00:00, ?it/s]



Combining plots into one pdf:: 1it [00:00,  3.62it/s]



Combining plots into one pdf:: 2it [00:00,  3.76it/s]



Combining plots into one pdf:: 3it [00:01,  1.68it/s]



Combining plots into one pdf:: 4it [00:02,  1.68it/s]



Combining plots into one pdf

In [27]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
#external_stylesheets = [dbc.themes.LITERA]

app = dash.Dash(__name__, external_stylesheets = external_stylesheets)

tabs_init = []
for chart in charts:
    if info['charts'].loc[chart, 'chart_status'] == 'ON':
        tabs_init.append(dcc.Tab(label=info['charts']['chart_label'][chart], value="".join(["tab-", str(chart)])))

diff = datetime.now(timezone('UTC')) - date_end
last_date = humanize.naturaldelta(diff)
update_text = html.Div(html.P('Data last retrieved ' + last_date + ' ago'))

app.layout = html.Div(children=[
    html.Div([html.Img(src=app.get_asset_url('ToOL-PRO-BES_2.png'), style={'width':'90%', 'max-width': '100%'})], style={'textAlign': 'center'}),
    update_text,
    html.Div(children=[
        dcc.Tabs(id="tabs",
            value="tab-0",
            children=tabs_init),

        html.Div(children=[
            dcc.Loading(id='tabs-content')
        ]),
    ]),
])

@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    time.sleep(2)
    if tab == 'tab-0':
        return html.Div(id='loading-0', children=dcc_chart_figs[0])
    elif tab == 'tab-1':
        return html.Div(id='loading-1', children=dcc_chart_figs[1])
    elif tab == 'tab-2':
        return html.Div(id='loading-2', children=dcc_chart_figs[2])
    elif tab == 'tab-3':
        return html.Div(id='loading-3', children=dcc_chart_figs[3])

In [28]:
if __name__ == '__main__':
    app.run_server()
    #debug=True, dev_tools_hot_reload_interval=5000)
                   #dev_tools_hot_reload_max_retry=30)

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Mar/2020 09:38:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2020 09:38:37] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2020 09:38:37] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2020 09:38:37] "GET /_favicon.ico?v=1.9.1 HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2020 09:38:38] "GET /assets/ToOL-PRO-BES_2.png HTTP/1.1" 304 -
127.0.0.1 - - [02/Mar/2020 09:38:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2020 09:39:55] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
tab1_content = dbc.Card(
    dbc.CardBody(
        [
            html.P("This is tab 1!", className="card-text"),
            dbc.Button("Click here", color="success"),
        ]
    ),
    className="mt-3",
)

tab2_content = dbc.Card(
    dbc.CardBody(
        [
            html.P("This is tab 2!", className="card-text"),
            dbc.Button("Don't click here", color="danger"),
        ]
    ),
    className="mt-3",
)


tabs = dbc.Tabs(
    [
        dbc.Tab(tab1_content, label="Tab 1"),
        dbc.Tab(tab2_content, label="Tab 2"),
        dbc.Tab(
            "This tab's content is never seen", label="Tab 3", disabled=True
        ),
    ]
)

layout = html.Div([tabs])

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LITERA]

app.layout = layout